<a href="https://colab.research.google.com/github/soham-chitnis10/SAiDL-assignment/blob/main/Computer%20Vision/Semi_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torch.utils.data import Dataset
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
#data augmentation and Normalization
train_transform = transforms.Compose([
        transforms.Resize((272,272)),
        transforms.RandomRotation(15,),
        transforms.RandomCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
    ])

test_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.507, 0.487, 0.441], std=[0.267, 0.256, 0.276])
    ])

trainset = datasets.STL10(
    root="data",
    split='train',
    download=True,
    transform=train_transform
)
testset = datasets.STL10(
    root="data",
    split='test',
    download=True,
    transform=test_transform
)
unlabeledset = datasets.STL10(
    root="data",
    split='unlabeled',
    download=True,
    transform=train_transform
)
from torch.utils.data import DataLoader
train_loader = DataLoader(trainset, batch_size=64, shuffle=True,num_workers = 2)
test_loader = DataLoader(testset, batch_size=64, shuffle=True,num_workers = 2)
unlabeled_loader = DataLoader(unlabeledset, batch_size=32, shuffle=True,num_workers = 2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Num_class=10
import torch.nn.functional as F

  0%|          | 0/2640397119 [00:00<?, ?it/s]

Extracting data/stl10_binary.tar.gz to data
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model_ft = models.resnet18().to(device)

In [ ]:
def evaluate(model, test_loader,criterion):
    model.eval()
    correct = 0 
    loss = 0
    with torch.no_grad():
        for data, labels in test_loader:
            data = data.to(device)
            output = model(data)
            predicted = torch.max(output,1)[1]
            correct += (predicted == labels.to(device)).sum()
            loss += criterion(output, labels.to(device)).item()

    return (float(correct)/len(testset)) *100, (loss/len(test_loader))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = torch.optim.Adam( model_ft.parameters(), lr = 0.001)

In [ ]:
from tqdm.notebook import tqdm
def train_supervised(model, train_loader, test_loader,optimizer,criterion,EPOCHS):
    model.train()
    for epoch in tqdm(range(EPOCHS)):
        torch.cuda.empty_cache()
        correct = 0
        running_loss = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            output = model(X_batch)
            labeled_loss = criterion(output, y_batch)
                       
            optimizer.zero_grad()
            labeled_loss.backward()
            optimizer.step()
            running_loss += labeled_loss.item()
        
        if epoch %10 == 0:
            test_acc, test_loss = evaluate(model, test_loader,criterion)
            print('Epoch: {} : Train Loss : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, running_loss/(10 * len(trainset)), test_acc, test_loss))
            model.train()

In [ ]:
train_supervised(model_ft, train_loader, test_loader,optimizer_ft,criterion,100)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: 0 : Train Loss : 0.00172 | Test Acc : 30.75000 | Test Loss : 1.768 
Epoch: 10 : Train Loss : 0.00082 | Test Acc : 53.93750 | Test Loss : 1.266 
Epoch: 20 : Train Loss : 0.00060 | Test Acc : 60.35000 | Test Loss : 1.135 
Epoch: 30 : Train Loss : 0.00043 | Test Acc : 68.02500 | Test Loss : 0.969 
Epoch: 40 : Train Loss : 0.00036 | Test Acc : 70.43750 | Test Loss : 0.940 
Epoch: 50 : Train Loss : 0.00021 | Test Acc : 73.72500 | Test Loss : 0.868 
Epoch: 60 : Train Loss : 0.00015 | Test Acc : 70.20000 | Test Loss : 1.147 
Epoch: 70 : Train Loss : 0.00009 | Test Acc : 72.32500 | Test Loss : 1.041 
Epoch: 80 : Train Loss : 0.00011 | Test Acc : 75.22500 | Test Loss : 0.978 
Epoch: 90 : Train Loss : 0.00012 | Test Acc : 74.36250 | Test Loss : 1.126 



In [ ]:
test_acc, test_loss = evaluate(model_ft, test_loader,criterion)
print('Test Acc : {:.5f} | Test Loss : {:.3f} '.format(test_acc, test_loss))
torch.save(model_ft.state_dict(), 'supervised_weights')

Test Acc : 73.56250 | Test Loss : 1.325 


In [ ]:
model_ft.load_state_dict(torch.load('supervised_weights'))

<All keys matched successfully>

In [ ]:
T1 = 100
T2 = 1000
af = 3

def alpha_weight(epoch):
    if epoch < T1:
        return 0.0
    elif epoch > T2:
        return af
    else:
         return ((epoch-T1) / (T2-T1))*af

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from tqdm.notebook import tqdm

acc_scores = []
unlabel = []
pseudo_label = []

alpha_log = []
test_acc_log = []
test_loss_log = []

def semisup_train(model, train_loader, unlabeled_loader, test_loader,optimizer,EPOCHS,criterion):
    
    # Instead of using current epoch we use a "step" variable to calculate alpha_weight
    # This helps the model converge faster
    step = 100 
    best_accuracy = 0
    model.train()
    for epoch in tqdm(range(EPOCHS)):
        torch.cuda.empty_cache()
        for batch_idx, x_unlabeled in enumerate(unlabeled_loader):
            
            
            # Forward Pass to get the pseudo labels
            x_unlabeled = x_unlabeled[0].to(device)
            model.eval()
            output_unlabeled = model(x_unlabeled)
            _, pseudo_labeled = torch.max(output_unlabeled, 1)
            model.train()
            
            
            # Now calculate the unlabeled loss using the pseudo label
            output = model(x_unlabeled)
            unlabeled_loss = alpha_weight(step) * criterion(output, pseudo_labeled)   
            
            # Backpropogate
            optimizer.zero_grad()
            unlabeled_loss.backward()
            optimizer.step()
            
            
            # For every 200 batches train one epoch on labeled data 
            if batch_idx % 200 == 0:
                
                # Normal training procedure
                for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
                    X_batch = X_batch.to(device)
                    y_batch = y_batch.to(device)
                    output = model(X_batch)
                    labeled_loss = criterion(output, y_batch)

                    optimizer.zero_grad()
                    labeled_loss.backward()
                    optimizer.step()
                
                # Now we increment step by 1
                step += 1
                

        test_acc, test_loss =evaluate(model, test_loader,criterion)
        print('Epoch: {} : Alpha Weight : {:.5f} | Test Acc : {:.5f} | Test Loss : {:.3f} '.format(epoch, alpha_weight(step), test_acc, test_loss))
        if test_acc > best_accuracy :
          best_accuracy = test_acc
          torch.save(model.state_dict(), 'semi_supervised_weights')
        """ LOGGING VALUES """
        alpha_log.append(alpha_weight(step))
        test_acc_log.append(test_acc/100)
        test_loss_log.append(test_loss)
        """ ************** """
        model.train()

In [ ]:
semisup_train(model_ft, train_loader, unlabeled_loader, test_loader,optimizer_ft,10,criterion)

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 : Alpha Weight : 0.05333 | Test Acc : 34.73750 | Test Loss : 4.993 
Epoch: 1 : Alpha Weight : 0.10667 | Test Acc : 40.26250 | Test Loss : 4.459 
Epoch: 2 : Alpha Weight : 0.16000 | Test Acc : 45.07500 | Test Loss : 4.280 
Epoch: 3 : Alpha Weight : 0.21333 | Test Acc : 59.50000 | Test Loss : 2.132 
Epoch: 4 : Alpha Weight : 0.26667 | Test Acc : 27.03750 | Test Loss : 6.819 
Epoch: 5 : Alpha Weight : 0.32000 | Test Acc : 54.71250 | Test Loss : 2.986 


In [ ]:
model_ft.load_state_dict(torch.load('semi_supervised_weights'))

<All keys matched successfully>

In [ ]:
test_acc, test_loss = evaluate(model_ft, test_loader,criterion)
print('Test Acc : {:.5f} | Test Loss : {:.3f} '.format(test_acc, test_loss))

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Test Acc : 59.50000 | Test Loss : 2.132 
